<a href="https://colab.research.google.com/github/edumntg/OPF-python/blob/main/Pyomo_OPF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pyomo

     |████████████████████████████████| 9.2 MB 5.1 MB/s 
     |████████████████████████████████| 49 kB 3.1 MB/s 


In [2]:
!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

In [3]:
from pyomo.environ import *
import numpy as np
from math import pi

# Data

In [172]:
Sbase = 10 # MW

buses = {
    1: [1, 0, 1.00, 0.0, 0.0, 0.0, 0.0, 0.0],
    2: [2, 1, 1.01, 0.0, 0.0, 0.0, 0.0, 0.0],
    3: [3, 2, 1.00, 0.0, 0.0, 0.0, 0.3, 0.1]
}

lines = {
    1: [1, 2, 0.0192, 0.0575, 0.0264, 1, 30/Sbase],
    2: [1, 3, 0.0452, 0.1852, 0.0204, 1, 30/Sbase],
    3: [2, 3, 0.0570, 0.1737, 0.0184, 1, 30/Sbase]
}

gens = {
    1: [1, 0/Sbase, 20/Sbase, -20/Sbase, 100/Sbase, 0.00375, 2, 0],
    2: [2, 0/Sbase, 20/Sbase, -20/Sbase, 100/Sbase, 0.0175, 1.75, 0]
}

# Create Ybus

In [173]:
nb = len(buses)
nl = len(lines)
ng = len(gens)

Ybus = np.zeros((nb, nb), dtype=np.complex128)
g = np.zeros((nb, nb))
b = np.zeros((nb,nb))
# Loop through lines
for lineid, linedata in lines.items():
  i = linedata[0]-1
  k = linedata[1]-1
  Z = linedata[2] + 1j*linedata[3]
  Bs = 1j*linedata[4]
  a = linedata[5]

  Ybus[i][i] += (1/(Z*a**2))
  Ybus[k][k] += (1/(Z*a**2))

  Ybus[i][i] += Bs
  Ybus[k][k] += Bs

  Ybus[i][k] -= 1/(a*Z)
  Ybus[k][i] -= 1/(a*Z)

  b[i][k] = Bs.imag
  b[k][i] = Bs.imag

G = Ybus.real
B = Ybus.imag

print(Ybus)
print(G)
print(B)

[[ 6.46838347-20.69594776j -5.22464618+15.64672684j
  -1.24373729 +5.09602092j]
 [-5.22464618+15.64672684j  6.9301765 -20.79930607j
  -1.70553032 +5.19737923j]
 [-1.24373729 +5.09602092j -1.70553032 +5.19737923j
   2.9492676 -10.25460015j]]
[[ 6.46838347 -5.22464618 -1.24373729]
 [-5.22464618  6.9301765  -1.70553032]
 [-1.24373729 -1.70553032  2.9492676 ]]
[[-20.69594776  15.64672684   5.09602092]
 [ 15.64672684 -20.79930607   5.19737923]
 [  5.09602092   5.19737923 -10.25460015]]


# Objective Function

In [174]:
def ObjectiveFunction(model):
  Cost = 0.0
  for genid, gendata in gens.items():
    bus = gendata[0]
    a = gendata[4]
    b = gendata[5]
    c = gendata[6]

    Cost += c*model.Pgen[bus]**2 + b*model.Pgen[bus] + a

  return Cost
  #return -sum(model.l[i] for i in model.bus)

# Constraints

In [189]:
def MinGen_P(model, bus):
  keys = [key for (key, v) in gens.items() if v[0] == bus]
  lb = 0
  if keys:
    lb = gens[keys[0]][1]
    
  return model.Pgen[bus] >= lb
  
def MaxGen_P(model, bus):
  keys = [key for (key, v) in gens.items() if v[0] == bus]
  ub = 0
  if keys:
    ub = gens[keys[0]][2]
    
  return model.Pgen[bus] <= ub

def MinGen_Q(model, bus):
  keys = [key for (key, v) in gens.items() if v[0] == bus]
  lb = 0
  if keys:
    lb = gens[keys[0]][3]
    
  return model.Qgen[bus] >= lb
  
def MaxGen_Q(model, bus):
  keys = [key for (key, v) in gens.items() if v[0] == bus]
  ub = 0
  if keys:
    ub = gens[keys[0]][4]
    
  return model.Qgen[bus] <= ub

def MaxFlowLineik(model, line):
  S = lines[line][6]
  i = lines[line][0]
  k = lines[line][1]
  return model.Pflow[i,k]**2+model.Qflow[i,k]**2 <= S

def MaxFlowLineki(model, line):
  S = lines[line][6]
  i = lines[line][1]
  k = lines[line][0]
  return model.Pflow[i,k]**2+model.Qflow[i,k]**2 <= S

def KirchoffBusesP(model, bus):
  Pik = 0
  Pgbus = model.Pgen[bus]
    
  for linea in model.line:
    i = lines[linea][0]
    if i == bus: # gen id is the same as bus id
      j = lines[linea][1]
      Pik += model.Pflow[i,j]
      
  for linea in model.line:
    i = lines[linea][1]
    if i == bus: # gen id is the same as bus id
      j = lines[linea][0]
      Pik += model.Pflow[i,j]
    
  return Pgbus == buses[bus][6] + Pik

def KirchoffBusesQ(model, bus):
  Qik = 0
  Qgbus = model.Qgen[bus]
  Qshunt = 0
  for linea in model.line:
    i = lines[linea][0]
    if i == bus: # gen id is the same as bus id
      j = lines[linea][1]
      Qik += model.Qflow[i,j]
      
  for linea in model.line:
    i = lines[linea][1]
    if i == bus: # gen id is the same as bus id
      j = lines[linea][0]
      Qik += model.Qflow[i,j]
      
    
  return Qgbus == buses[bus][7] + Qik + Qshunt
  
# Lines equations
def PflowEq1(model, linea):
  i = lines[linea][0]
  j = lines[linea][1]
  return model.Pflow[i, j] == (-G[i-1][j-1] + g[i-1][j-1])*model.V[i]**2 + model.V[i]*model.V[j]*(G[i-1][j-1]*cos(model.theta[i]-model.theta[j]) + B[i-1][j-1]*sin(model.theta[i]-model.theta[j]))

def PflowEq2(model, linea):
  i = lines[linea][1]
  j = lines[linea][0]
  return model.Pflow[i, j] == (-G[i-1][j-1] + g[i-1][j-1])*model.V[i]**2 + model.V[i]*model.V[j]*(G[i-1][j-1]*cos(model.theta[i]-model.theta[j]) + B[i-1][j-1]*sin(model.theta[i]-model.theta[j]))

def QflowEq1(model, linea):
  i = lines[linea][0]
  j = lines[linea][1]
  return model.Qflow[i, j] == (B[i-1][j-1] - b[i-1][j-1])*model.V[i]**2 + model.V[i]*model.V[j]*(-B[i-1][j-1]*cos(model.theta[i]-model.theta[j]) + G[i-1][j-1]*sin(model.theta[i]-model.theta[j]))

def QflowEq2(model, linea):
  i = lines[linea][1]
  j = lines[linea][0]
  return model.Qflow[i, j] == (B[i-1][j-1] - b[i-1][j-1])*model.V[i]**2 + model.V[i]*model.V[j]*(-B[i-1][j-1]*cos(model.theta[i]-model.theta[j]) + G[i-1][j-1]*sin(model.theta[i]-model.theta[j]))


# Finally, solve

In [190]:
model = ConcreteModel()

model.bus = Set(initialize = buses.keys())
model.line = Set(initialize = lines.keys())
model.gen = Set(initialize = gens.keys())

# Create variables
model.Pgen = Var(model.bus, initialize = 0)
model.Qgen = Var(model.bus, initialize = 0)
model.V = Var(model.bus, initialize = 0.0, bounds = (0.9, 1.1), within = NonNegativeReals)
model.theta = Var(model.bus, initialize = 0.0, bounds = (-pi, pi))

# Line flows
model.Pflow = Var(model.bus, model.bus)
model.Qflow = Var(model.bus, model.bus)

model.obj = Objective(rule = ObjectiveFunction, sense = minimize)

model.c0 = Constraint(expr = model.theta[1] == 0)

model.c1 = Constraint(model.bus, rule = KirchoffBusesP)
model.c2 = Constraint(model.bus, rule = KirchoffBusesQ)

model.c3 = Constraint(model.bus, rule = MaxGen_P)
model.c4 = Constraint(model.bus, rule = MinGen_P)
model.c5 = Constraint(model.bus, rule = MaxGen_Q)
model.c6 = Constraint(model.bus, rule = MinGen_Q)


model.c7 = Constraint(model.line, rule = MaxFlowLineik)
model.c8 = Constraint(model.line, rule = MaxFlowLineki)

model.c9 = Constraint(model.line, rule = PflowEq1)
model.c10 = Constraint(model.line, rule = PflowEq2)
model.c11 = Constraint(model.line, rule = QflowEq1)
model.c12 = Constraint(model.line, rule = QflowEq2)


# Solve

In [191]:
solver = SolverFactory('ipopt', executable='/content/ipopt')
results = solver.solve(model)
print(results.solver.termination_condition)

optimal


In [192]:
def PrintOPFACResults(model, buses, lineas, gens, shunts):

	nb = len(buses)
	nl = len(lineas)
	ng = len(gens)
	ns = len(shunts)

	print('BusID	V	th	Pg	Qg	l	Pl	Ql	Qshunt\n')
	l = {}
	for i in model.bus:
		Qshunt = 0
		Pg = abs(model.Pgen[i]())
		Qg = model.Qgen[i]()
		
		print("{0:.0f}	{1:.4f}	{2:.4f}	{3:.4f}	{4:.4f}	{5:.4f}	{6:.4f}	{7:.4f}	{8:.4f}".format(i,model.V[i](),model.theta[i](),Pg,Qg, 1.0, buses[i][6], buses[i][7], 0.0))

	Pgtotal = sum(model.Pgen[i]() for i in model.bus)
	Qgtotal = sum(model.Qgen[i]() for i in model.bus)

	Ploadtotal = sum(buses[i][6] for i in model.bus)
	Qloadtotal = sum(buses[i][7] for i in model.bus)

	print("\n")
	print("TOTAL			{0:.4f}	{1:.4f}		{2:.4f}	{3:.4f}".format(Pgtotal, Qgtotal, Ploadtotal, Qloadtotal))
	print("\n\n")

	print("Busi	Busk	Pik	Pki	Qik	Qki")
	Pik = np.zeros((nb,nb))
	Pki = np.zeros((nb,nb))
	Qik = np.zeros((nb,nb))
	Qki = np.zeros((nb,nb))

	for l in model.line:
		i = lineas[l][0]
		j = lineas[l][1]
		print("{0:.0f}	{1:.0f}	{2:.4f}	{3:.4f}	{4:.4f}	{5:.4f}".format(i,j,model.Pflow[i,j](),model.Pflow[j,i](),model.Qflow[i,j](),model.Qflow[j,i]()))
		
	Ploss = 0
	Qloss = 0
	for l in model.line:
		i = lineas[l][0]
		j = lineas[l][1]
		Ploss += model.Pflow[i,j]() + model.Pflow[j,i]()
		Qloss += model.Qflow[i,j]() + model.Qflow[j,i]()


	Pl_supplied = sum(buses[i][6] for i in model.bus)
	Pl_total = sum(buses[i][6] for i in model.bus)
	perc_supplied = (Pl_supplied/Pl_total)*100
	
	print("\n")
	print("Total Ploss: {0:.4f}\nTotal Qloss: {1:.4f}".format(Ploss,Qloss))
	print("Total Load Supplied: {0:.4f}%".format(perc_supplied))

In [193]:
PrintOPFACResults(model, buses, lines, gens, [])

BusID	V	th	Pg	Qg	l	Pl	Ql	Qshunt

1	1.1000	-0.0000	0.1428	-0.0203	1.0000	0.0000	0.0000	0.0000
2	1.1000	0.0002	0.1592	-0.0294	1.0000	0.0000	0.0000	0.0000
3	1.0882	-0.0213	0.0000	-0.0000	1.0000	0.3000	0.1000	0.0000


TOTAL			0.3020	-0.0497		0.3000	0.1000



Busi	Busk	Pik	Pki	Qik	Qki
1	2	-0.0033	0.0033	-0.0314	-0.0324
1	3	0.1461	-0.1453	0.0111	-0.0565
2	3	0.1559	-0.1547	0.0030	-0.0435


Total Ploss: 0.0020
Total Qloss: -0.1497
Total Load Supplied: 100.0000%
